In [1]:
import warnings


warnings.filterwarnings('ignore')

# Advanced Python

## Python Data Model

In [3]:
obj = NotImplemented
print(f"Object id: {id(obj)}")
print(f"Object type: {type(obj)}")
print(f"Object value: {obj}") # actually a string representation of the object
cols = 5
print("\n".join([" ".join(method for method in dir(obj)[line*cols: (line+1)*cols]) for line in range(len(dir(obj))//cols + 1)])) # do not do that at home!

Object id: 140713537457360
Object type: <class 'NotImplementedType'>
Object value: NotImplemented
__class__ __delattr__ __dir__ __doc__ __eq__
__format__ __ge__ __getattribute__ __gt__ __hash__
__init__ __init_subclass__ __le__ __lt__ __ne__
__new__ __reduce__ __reduce_ex__ __repr__ __setattr__
__sizeof__ __str__ __subclasshook__


### Special (magic, dunder) methods

- Implement certain operations that are invoked by special syntax or function
- Python’s approach to operator overloading
- Setting to `None` indicates that the corresponding operation is not available

Examples:
- `__new__`- called to create a new instance of a class
- `__init__` - called after the instance has been created, but before returning
- `__del__` - called when the instance is about to be deleted

### `__str__` and `__repr__`

In [4]:
s = "I am a string."
s_str = str(s)
s_repr = repr(s)
s_str, s_repr

('I am a string.', "'I am a string.'")

If at all possible, `__repr__` should look like a valid Python expression that could be used to recreate an object with the same value (given an appropriate environment).

In [5]:
print(f"repr: {eval(s_repr)}")
print(f"str: {eval(s_str)}")

repr: I am a string.


SyntaxError: invalid syntax (<string>, line 1)

### Ordering

- `__eq__` $\Longleftrightarrow$ `==`
- `__ne__` $\Longleftrightarrow$ `!=`
- `__lt__` $\Longleftrightarrow$ `<`
- `__gt__` $\Longleftrightarrow$ `>`
- `__le__` $\Longleftrightarrow$ `<=`
- `__ge__` $\Longleftrightarrow$ `>=`

### Arithmetic Operations
- `__add__(self, other)` $\Longleftrightarrow$ `self + other`
- `__radd__(self, other)` $\Longleftrightarrow$ `other + self`
- `__iadd__(self, other)` $\Longleftrightarrow$ `self += other`
```
...and others
```

In [6]:
class OrderedComplex(complex):        
    def __eq__(self, other):
        if isinstance(other, OrderedComplex):
            return (self * self.conjugate()).real == (other * other.conjugate()).real
    
    def __lt__(self, other):
        if isinstance(other, OrderedComplex):
            return (self * self.conjugate()).real < (other * other.conjugate()).real
    
    def __le__(self, other):
        if isinstance(other, OrderedComplex):
            return (self * self.conjugate()).real <= (other * other.conjugate()).real


a = OrderedComplex(1 + 2j)
b = OrderedComplex(2 + 1j)

a < b, a <= b, a == b, a >=b, a > b

(False, True, True, True, False)

In [7]:
import functools

??functools.total_ordering

### Attribute access

- `__getattr__` and `__getattribute__`
- `__setattr__`
- `__delattr__`

`__getattr__` called when the default attribute access fails with an `AttributeError`

In [8]:
class Layer:
    def __init__(self):
        self._params = {}
    
    def __setattr__(self, key, value):
        if not key.startswith("_"):
            self._params[key]=value
        super().__setattr__(key, value)
    
    @property
    def parameters(self):
        return self._params
        

class LinearLayer(Layer):
    def __init__(self, weights, bias):
        super().__init__()
        
        self.weights = weights
        self.bias = bias
        
        
layer_1 = LinearLayer([[1, 2], [3, 4]], [0, 1])

for key, value in layer_1.parameters.items():
    print(f"{key}: {value}")

weights: [[1, 2], [3, 4]]
bias: [0, 1]


## Containers

Containers usually are **sequences** (such as lists or tuples) or **mappings** (like dictionaries), but can represent other containers as well.

- `__len__`
```
CPython implementation detail: In CPython, the length is required to be at most sys.maxsize.
```
- `__length_hint__` # called by `operator.length_hint(obj, default=0)`
```
Return an estimated length for the object o. First try to return its actual length, then an estimate using object.__length_hint__(), and finally return the default value.
```

In [9]:
import sys


print(f"{sys.maxsize}, {sys.maxsize == 2**63 - 1}")

len(range(2**63))

9223372036854775807, True


OverflowError: Python int too large to convert to C ssize_t

## Containers

- `__getitem__`, `__setitem__`, `__delitem__`, `__missing__`

Implements `self[key]`

- `__iter__` and `__reversed__`

If the `__reversed__` method is not provided, the reversed() built-in will fall back to using the sequence protocol (`__len__` and `__getitem__`)

- `__contains__(self, item)` $\Longleftrightarrow$ `item in self`

How to create an immutable container?

```
__len__ + __getitem__
```

In [10]:
class ListContainer():
    def __init__(self, items): self.items = self.listify(items)
    def __bool__(self): return bool(self.items)
    def __getitem__(self, idx):
        if isinstance(idx, (int,slice)): return self.items[idx]
        if isinstance(idx[0], bool):
            assert len(idx)==len(self) # bool mask
            return [obj for include, obj in zip(idx, self.items) if include]
        return [self.items[i] for i in idx]
    def __len__(self): return len(self.items)
    def __iter__(self): return iter(self.items)
    def __setitem__(self, i, obj): self.items[i] = obj
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self):
        res = f'{self.__class__.__name__} ({len(self)} items)\n{self.items[:10]}'
        if len(self)>10: res = res[:-1]+ '...]'
        return res
    
    @staticmethod
    def listify(obj):
        if obj is None: return []
        if isinstance(obj, list): return obj
        if isinstance(obj, (str, int, float, complex)): return [obj]
        if isinstance(obj, Iterable): return list(obj)

## Callable

In [11]:
import random


class Die:
    def __init__(self, num_of_faces=6):
        self.faces = range(1, num_of_faces+1)
        
    def __call__(self):
        return random.choice(self.faces)
    
    def __repr__(self):
        return f"Die({len(self.faces)})"

    
d24 = Die(24)
d24(), d24(), d24()

(9, 22, 22)

## List of dice

In [12]:
class Dice(ListContainer):
    def __init__(self, dice_list):
        super().__init__(dice_list)
        
    def __call__(self):
        return [die() for die in self]

In [13]:
d_pack = Dice([Die(6), Die(18), Die(24)])

print(d_pack)
bool_idx = [len(die.faces) <= 18 for die in d_pack]
print(bool_idx)
print(d_pack[bool_idx])
d_pack()

Dice (3 items)
[Die(6), Die(18), Die(24)]
[True, True, False]
[Die(6), Die(18)]


[4, 10, 18]

## Iterators and Iterable

An object representing a stream of data.

### Iterator protocol

- `__iter__` - return the iterator object itself
- `__next__` - return the next item from the iterator

Return the next item from the container. If there are no further items, raise the `StopIteration` exception. Once an iterator’s `__next__` method raises `StopIteration`, it must continue to do so on subsequent calls.

**Iterable** - object that returns an iterator.


$$
iterators \supset iterable
$$

$$
iterators \subset iterable
$$


In [14]:
s = "string"
s_iter = iter(s)

# print(list(zip(s, s, s)))
# print(list(zip(s_iter, s_iter, s_iter)))
# print(list(zip(iter(s), iter(s), iter(s))))
# print(type(s_iter))

In [15]:
import collections


class DiceIterator(collections.abc.Iterator):
    def __init__(self, dice, cursor=-1):
        self.dice = dice
        self._cursor = cursor
        
    def __next__(self):
        self._cursor += 1
        if self._cursor >= len(self.dice):
            raise StopIteration
        return self.dice[self._cursor]
    

class DiceCollection(collections.abc.Iterable):
    def __init__(self, list_of_faces):
        self.dice = [Die(num_of_faces) for num_of_faces in list_of_faces]
    
    def __iter__(self):
        return DiceIterator(self.dice, -1)

In [16]:
d_pack = DiceCollection([6, 8, 9, 4])

print(type(d_pack))

d_iter = iter(d_pack)
print(type(d_iter))
d_iter_2 = iter(d_iter)
print(type(d_iter_2))

<class '__main__.DiceCollection'>
<class '__main__.DiceIterator'>
<class '__main__.DiceIterator'>


In [17]:
??DiceIterator.__iter__

In [18]:
for die in DiceCollection([6, 8, 9, 4]):
    print(die(), end=", ")
    

2, 4, 6, 2, 

## Iterators form `__getitem__`

In [19]:
class HiddenList:
    def __init__(self, lst):
        self.lst = lst


h_list = HiddenList([1, 2, 3])
iter(h_list)

TypeError: 'HiddenList' object is not iterable

In [20]:
class IterableHiddenList(HiddenList):
    def __getitem__(self, i):
        return self.lst[i]
    

ih_list = IterableHiddenList([1, 2, 3])
iter(ih_list)

for i in ih_list:
    print(i)

1
2
3


## Generators

Python’s generators provide a convenient way to implement the iterator protocol.

Generator is a **function** which returns a _generator iterator_.

In [21]:
def fib():
    a, b = 0, 1
    print("Here!")
    while True:
        yield b
        a, b = b, a + b

print(type(fib))
fib_gen = fib()
print(type(fib_gen))

for _, i in zip(range(50), fib_gen):
    print(i, end=", ")
print(next(fib_gen))

<class 'function'>
<class 'generator'>
Here!
1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074


## Generators

A generator function can also contain return statements of the form: `return`

Each `yield` temporarily suspends processing, remembering the location execution state (including local variables and pending try-statements). When the generator iterator resumes, it picks up where it left off (in contrast to functions which start fresh on every invocation)

In [22]:
def do_in_order():
    x = 1
    print(f"Do first, {x}")
    yield
    x += 1
    print(f"Do second, {x}")
    yield
    x *= x
    print(f"Do third, {x}")

gen = do_in_order()
next(gen)
next(gen)
next(gen, None)

Do first, 1
Do second, 2
Do third, 4


### Send

In [23]:
def do_in_order_2():
    x = 1
    print(f"Do first, {x}")
    x = yield
    print(f"Do second, {x}")
    x = yield 42
    print(f"Do third, {x}")

gen = do_in_order_2()
for _ in gen:
    pass

Do first, 1
Do second, None
Do third, None


In [24]:
gen = do_in_order_2()
gen.send(None)

Do first, 1


In [25]:
out = gen.send("I am in generator!")
print(out)

Do second, I am in generator!
42


In [26]:
try:
    out = gen.send("lol")
except StopIteration:
    print("I'm out!")

Do third, lol
I'm out!


### Throw

In [27]:
def g():
    try:
        yield 42
    except Exception as e:
        yield e

gen = g()
next(gen)

42

In [28]:
gen.throw(NotImplementedError, "WTF?!")

NotImplementedError('WTF?!')

In [29]:
# gen.throw(NotImplementedError, "WTF?!")

### Close

In [30]:
gen = do_in_order_2()

gen.send(None)

Do first, 1


In [31]:
gen.close()

In [32]:
gen.send("Smth")

StopIteration: 

## Coroutines (via generators)

- generalization of subroutines
- can be entered, exited, and resumed at many different points
- from Python 3.7 can be implement with `async` and `await` syntax

In [33]:
def check_number(number):
    print(f"Ready for check {number}s")
    while True:
        new_number = yield
        if number == new_number:
            print(f"Got {number}!")

In [34]:
is_six = check_number(6)
d6 = Die(6)

# initialize the generator
is_six.send(None)

for _ in range(30):
    is_six.send(d6())

Ready for check 6s
Got 6!
Got 6!
Got 6!
Got 6!


## yield from

- pass the execution to another generator
- pass `send` and `throw`

In [35]:
def chain(*iterables):
    for iterable in iterables:
        yield from iterable

In [36]:
i1, i2, i3, i4, i5 = (
    [1, 2], ("4", "5"), {"key1": "val1", "key2": "val2"}, "iter", {("a",), ("b",)}
)

list(chain(i1, i2, i3, i4, i5))

[1, 2, '4', '5', 'key1', 'key2', 'i', 't', 'e', 'r', ('b',), ('a',)]

## Context Manager

- `__enter__(self)`
- `__exit__(self, exception_type, exception_value, traceback)`

Patterns:
- acquisition/release of the resource
- doing something in different context

In [37]:
import os


class cd:
    def __init__(self, path):
        self.path = path
        
    def __enter__(self):
        self.cwd = os.getcwd()
        os.chdir(self.path)
        
    def __exit__(self, *args):
        os.chdir(self.cwd)
        

print(os.getcwd())
with cd(".."):
    print(os.getcwd())

C:\Users\Artur_Paniukov\Documents\Python\lectures\advanced_python
C:\Users\Artur_Paniukov\Documents\Python\lectures


## Context Manager

```
with resource_1() as r1, \
     resource_2() as r2:
    
    some_work()
```

`__enter__` can return a resource.

If an `__exit__` returns `True`, than an exception will be suppressed.

## Context Manager and Generators

In [38]:
from contextlib import contextmanager


@contextmanager
def cd_gen(path):
    cwd = os.getcwd()
    try:
        # <__enter__>
        os.chdir(path)
        # </__enter__>
        yield
    finally:
        # <__exit__>
        os.chdir(cwd)
        # </__exit__>
        
print(os.getcwd())
with cd_gen(".."):
    print(os.getcwd())

C:\Users\Artur_Paniukov\Documents\Python\lectures\advanced_python
C:\Users\Artur_Paniukov\Documents\Python\lectures


# Bonus

In [39]:
def deco(*, func=None, **kwargs):
    if not func:
        return lambda f: deco(func=f, **kwargs)
        
    def inner(*args, **inner_kwargs):
        print(kwargs)
        return func(*args, **inner_kwargs)
    return inner
        

In [40]:
@deco(a=1, b=2)
def f():
    print(1)
    
f()

{'a': 1, 'b': 2}
1
